# Process one-group actinide cross section uncertainties for different libraries

In [1]:
import sandy

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

In [3]:
import pandas as pd

In [4]:
import logging
logging.getLogger().setLevel(logging.WARN)

In [5]:
nuclides = [922350, 922360, 922380, 932370, 932380, 942380, 942390, 942400, 942410, 942420, 952410, 952421, 952430, 962440]
libs = ["jeff_33", "jendl_40u", "endfb_80"]

In [6]:
# Options
iwt = 5  # PWR spectrum
ek = [1e-5, 2e7]  # 1 group
mt = [18, 102]  # two reactions: fission and capture

In [8]:
stds = []
for lib in libs:
    for zam in nuclides:
        nucl = sandy.zam.zam2nuclide(zam)
        print(f"processing nuclide {nucl} for library {lib.upper()}...")
        err = sandy.get_endf6_file(lib, "xs", zam).get_errorr(
            err=0.1, verbose=False, dryrun=False,
            xs=True, chi=False, nubar=False, mubar=False,
            errorr33_kws=dict(
                irespr=0,
                iwt=iwt,
                ek=ek,
                mt=mt,
            ),
        )
        if "errorr33" in err:
            std = err["errorr33"].get_cov().get_std().multiply(100).reset_index().assign(NUCL=nucl, LIB=lib)
            stds.append(std)
df = pd.concat(stds, ignore_index=True)

processing nuclide U235 for library JEFF_33...
processing nuclide U236 for library JEFF_33...
processing nuclide U238 for library JEFF_33...
processing nuclide Np237 for library JEFF_33...
processing nuclide Np238 for library JEFF_33...
processing nuclide Pu238 for library JEFF_33...
processing nuclide Pu239 for library JEFF_33...
processing nuclide Pu240 for library JEFF_33...
processing nuclide Pu241 for library JEFF_33...
processing nuclide Pu242 for library JEFF_33...
processing nuclide Am241 for library JEFF_33...
processing nuclide Am242m for library JEFF_33...
processing nuclide Am243 for library JEFF_33...
processing nuclide Cm244 for library JEFF_33...
processing nuclide U235 for library JENDL_40U...
processing nuclide U236 for library JENDL_40U...
processing nuclide U238 for library JENDL_40U...
processing nuclide Np237 for library JENDL_40U...
processing nuclide Np238 for library JENDL_40U...
processing nuclide Pu238 for library JENDL_40U...
processing nuclide Pu239 for libr

In [21]:
print(df.query("MT in @mt").to_string())

      MAT   MT                    E         STD    NUCL        LIB
0    9228   18  (1e-05, 20000000.0] 6.31499e-01    U235    jeff_33
1    9228  102  (1e-05, 20000000.0] 1.70583e+00    U235    jeff_33
2    9237   18  (1e-05, 20000000.0] 2.63840e+00    U238    jeff_33
3    9237  102  (1e-05, 20000000.0] 1.10201e+00    U238    jeff_33
4    9346   18  (1e-05, 20000000.0] 3.30412e+00   Np237    jeff_33
5    9346  102  (1e-05, 20000000.0] 4.58509e+00   Np237    jeff_33
6    9434   18  (1e-05, 20000000.0] 4.27258e+00   Pu238    jeff_33
7    9434  102  (1e-05, 20000000.0] 9.82995e+00   Pu238    jeff_33
8    9437   18  (1e-05, 20000000.0] 1.22453e+00   Pu239    jeff_33
9    9437  102  (1e-05, 20000000.0] 2.26990e+00   Pu239    jeff_33
10   9440   18  (1e-05, 20000000.0] 2.71615e+00   Pu240    jeff_33
11   9440  102  (1e-05, 20000000.0] 2.23112e+00   Pu240    jeff_33
12   9443   18  (1e-05, 20000000.0] 1.93646e+00   Pu241    jeff_33
13   9443  102  (1e-05, 20000000.0] 9.90368e+00   Pu241    jef